In [17]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [18]:
%store -r estSalary

In [19]:
credit = pd.read_csv('Credit.csv', usecols=list(range(1,12)))
credit_cp=credit.copy(deep=True)
credit_cp['Student_dummy'] = credit_cp.Student.map({'No':0, 'Yes':1})
credit_cp['Married_dummy'] = credit_cp.Married.map({'No':0, 'Yes':1})
credit_cp['Active'] = np.where(credit_cp['Balance']>0, 1,0)
credit_cp['HasCard']=np.where(credit_cp['Cards']>0, 1,0)
dummy=pd.get_dummies(credit_cp['Gender'])
credit_cp=pd.concat([credit_cp, dummy], axis=1)

In [20]:
X=credit_cp.iloc[:, [2,4,13,14]].values
y=credit_cp.iloc[:,0].values

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=42)

In [21]:
model_xgb=xgb.XGBRegressor(booster='gbtree')
model_xgb.fit(X_train, y_train)

[20:43:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [34]:
y_pred=model_xgb.predict(estSalary)
y_pred=y_pred[0]*10000

In [35]:
%store y_pred

Stored 'y_pred' (float64)


14.24387